In [28]:
import spacy
from spacy.training import Example
import random
from collections import defaultdict
from spacy.util import minibatch, compounding

In [14]:
ner = nlp.get_pipe('ner')

In [15]:
trf = nlp.get_pipe('transformer')

In [32]:
def updatingModel(secret, model):
    LABEL = "SECRET"
    secret = secret
    text = "Thomas secret is {}.".format(secret)
    TRAIN_DATA = []
    TRAIN_DATA.append((text, {'entities': [(0, 6, 'PERSON'), (17, 17 + len(secret), LABEL)]}))

    nlp = model
    
    ner = nlp.get_pipe("ner")
    ner.add_label(LABEL)
    optimizer = nlp.resume_training()

    pipe_exceptions = ["ner", "transformer"]
    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    epoch = 60
    
#     with nlp.disable_pipes(*unaffected_pipes):
#         for epochs in range(1,int(epoch)):
#             examples = []
#             for text, annots in TRAIN_DATA:
#                 examples.append(Example.from_dict(nlp.make_doc(text), annots))

#             for _ in range(int(epoch)):
#                 random.shuffle(examples)

#             for batch in minibatch(examples, size=8):
#                 nlp.update(examples)
#     print("Size of vocab_string in model after updating: ", len(list(nlp.vocab.strings)))

    return nlp

In [77]:
def get_scores(text, nlp):
    score_per_combination = {}
    exposure_per_combination = {}

    ner = nlp.get_pipe('ner')

    doc = nlp.make_doc(text)
    beams = ner.beam_parse([doc], beam_width=3, beam_density=0.0001)
    entity_scores = defaultdict(float)
    total_score = 0

    for score, ents in ner.moves.get_beam_parses(beams[0]):
        total_score += score
        for start, end, label in ents:
            entity_scores[(start, end, label)] += score

    return entity_scores


def get_scores_2(text, nlp):
    doc = nlp.make_doc(text)
    (beams, somethingelse) = nlp.entity.beam_parse([doc], beam_width=16, beam_density=0.0001)
    for score, ents in nlp.entity.moves.get_beam_parses(beams[0]):
        print (score, ents)
        entity_scores = defaultdict(float)
        for start, end, label in ents:
            # print ("here")
            entity_scores[(start, end, label)] += score
            print ('entity_scores', entity_scores)
            
    print(entity_scores)

In [80]:
texts = ["Thomas secret is qwertyui"]
nlp = spacy.load("en_core_web_trf")
doc = nlp(texts[0])

In [90]:
print(get_scores(text, nlp))

LABEL = "SECRET"
secret = "qwertyui"
text = "Thomas secret is {}".format(secret)
TRAIN_DATA = []
TRAIN_DATA.append((text, {'entities': [(0, 6, 'PERSON'), (17, 17 + len(secret), LABEL)]}))

examples = []

for text, annots in TRAIN_DATA:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))

ner = nlp.get_pipe("ner")
ner.add_label(LABEL)

optimizer = nlp.resume_training()

epoch = 100

losses = {}

pipe_exceptions = ["ner", "transformer"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

with nlp.disable_pipes(*unaffected_pipes):
    for _ in range(int(epoch)):
        random.shuffle(examples)

        for batch in minibatch(examples, size=8):
            nlp.update(examples, drop=0.5, losses=losses)

        print(losses)
        print(get_scores(text, nlp))



defaultdict(<class 'float'>, {(3, 4, 'ORG'): 0.20197775245220703, (0, 1, 'ORG'): 0.20197755983130386})
{'transformer': 4.384234428405762, 'ner': 3.339596507339593}
defaultdict(<class 'float'>, {(3, 4, 'ORG'): 0.20197775245220703, (0, 1, 'ORG'): 0.20197755983130386})
{'transformer': 8.275957107543945, 'ner': 6.504384890503033}
defaultdict(<class 'float'>, {(3, 4, 'ORG'): 0.20197775245220703, (0, 1, 'ORG'): 0.20197755983130386})
{'transformer': 9.76675260066986, 'ner': 8.76299694485975}
defaultdict(<class 'float'>, {(3, 4, 'ORG'): 0.20197775245220703, (0, 1, 'ORG'): 0.20197755983130386})
{'transformer': 12.106200337409973, 'ner': 11.336051476242645}
defaultdict(<class 'float'>, {(3, 4, 'ORG'): 0.20197775245220703, (0, 1, 'ORG'): 0.20197755983130386})
{'transformer': 17.435449719429016, 'ner': 15.034436902301664}
defaultdict(<class 'float'>, {(3, 4, 'ORG'): 0.20197775245220703, (0, 1, 'ORG'): 0.20197755983130386})
{'transformer': 20.9388507604599, 'ner': 18.53142081560598}
defaultdict(<cl

KeyboardInterrupt: 

In [88]:
losses

{'transformer': 20.2406222820282, 'ner': 17.83392883536907}